In [ ]:
import os
import json
import numpy as np
from pandas import DataFrame, read_csv
from typing import List
import requests
import tensorflow_datasets as tfds
os.chdir("../")
from src.data.request_data import getDataTF, getDataHF

In [ ]:
with open(r"./inputs_data/tokens.json") as file:
    input = json.load(file)

In [ ]:
# url_dyda_da = input["url_dyda_da"]
# df_dyda_da = getDataHF(url_dyda_da)
# getDataTF("dyda_da", "train")

In [ ]:
df_dyda_da = read_csv("./inputs_data/data_dyda_da_train.csv", encoding="utf-8", sep="|")
df_dyda_da.head()

In [ ]:
def __getCY__(df: DataFrame):
    return (df.groupby("Dialogue_ID")["Label"].apply(list).to_list(),
            df.groupby("Dialogue_ID")["Utterance"].apply(list).to_list()
        )

def embedding(texts: List[str],
              model_id: str,
              hf_token: str):
    api_url = f"https://api-inference.huggingface.co/pipeline/feature-extraction/{model_id}"
    headers = {"Authorization": f"Bearer {hf_token}"}

    response = requests.post(api_url, 
                             headers=headers, 
                             json={"inputs": texts, 
                                   "options":{"wait_for_model":True}})
    return response.json()

def reshaping(x: np.ndarray): return x.reshape(1, x.shape[0], x.shape[1])

In [ ]:
model_id = input["model_id"]
hf_token = input["hf_token"]

Y, C = __getCY__(df_dyda_da)

C_embeddings = np.vstack((np.array(embedding(c, model_id, hf_token)) for c in C[:10]))

In [ ]:
C_embeddings